## Construir kNN graph para un audio

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx 
import pandas as pd 

from pathlib import Path
from sklearn.neighbors import kneighbors_graph

In [ ]:
path_data = Path("directorio/con/parquet")

In [ ]:
labels = pd.read_parquet(
            path_data.joinpath("Deteccion-Fauna-P1_ECP_IV-test.parquet")
)
len(labels.audio.unique())

In [ ]:
fauna_by_audio = labels.groupby("audio")["fauna"].agg(sum)
fauna_by_audio[fauna_by_audio > 0].sort_values(ascending=False)

In [ ]:
data = pd.read_parquet(
            path_data.joinpath("Embeddings-P1_ECP_IV-test.parquet")
)
data.head()

In [ ]:
emb, metadata = data[:][data.columns[:1024]].to_numpy(), data[:][["audio","canal","fecha","tiempo"]]
list_audios = list(metadata.audio.unique())

In [ ]:
name_audio = "1687978462.8386_1687978522.8387.wav" # El con mas etiquetas de fauna: 9
num_audio = list_audios.index(name_audio)
idx_audio  = metadata[metadata.audio == list_audios[num_audio]].index
emb_audio  = emb[idx_audio.to_numpy(),:]

Crear kNN graph

In [ ]:
A = kneighbors_graph(emb_audio, n_neighbors=3)
n_nodes = A.toarray().shape[0]

Graficar

In [ ]:
rows,cols=np.where(A.toarray()==1)  # A.toarray() da la matriz de adyacencia
edges = [[r,c] for r,c in zip(rows,cols)]

In [ ]:
G = nx.Graph()
G.add_edges_from(edges)
print(G)

In [ ]:
## Plot
node_labels = list(labels.fauna[idx_audio.to_numpy()]) # etiquetas de los nodos (para colorear grafo)
pos_layout = nx.spring_layout(G, iterations=1000)

fig, ax= plt.subplots(1,1,figsize=(14,8),dpi=100)
nx.draw_networkx(   
    G, 
    pos=pos_layout,
    node_color = node_labels,  
    node_size=200,
    alpha=0.8,
    )